## Importing necessary packages

In [11]:
from sklearn.base import clone
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, LeaveOneOut, StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from typing import List

import numpy as np
import pandas as pd
import pickle
import umap
import warnings

warnings.filterwarnings("ignore")

In [12]:
EXPERIMENT = "OF"

DATASET_PATH = "../../legacy_data/12_classification_dataset/OF_2.csv"
LABELS_PATH  = "../../legacy_data/12_classification_dataset/video_anxiety_labels_for_ML_allLabels_May_2022.xlsx"

LABELS = [
    "High_Anxiety",
    "Middle_Anxiety",
    "Low_Anxiety"
]

## Data Preprocessing

In [13]:
def get_week_number(video_name: str) -> int:
    result = video_name.split('_')
    
    if len(result) == 6:
        return int(result[3][1])
    else:
        return int(result[2][1])

def get_video_set(video_name: str) -> str:
    result = video_name.split('_')
    return "Videos " + result[1][1]
    
def get_trial_number(video_name: str) -> int:
    result = video_name.split('_')
    return int(result[0][5:])

def preprocess_labels(labels: pd.DataFrame) -> pd.DataFrame:
    labels = labels.dropna()
    encodings = pd.get_dummies(labels["labels"])
    labels = pd.concat([labels, encodings], axis=1)
    labels = labels.drop(columns=["test", "labels"])
    labels = labels.rename(columns={
        "high anxiety": "High_Anxiety",
        "mid anxiety": "Middle_Anxiety",
        "low anxiety": "Low_Anxiety"
    })
    
    return labels

def check_na(df: pd.DataFrame):
    na_df = df.isna().sum()
    na_df = na_df[na_df > 0]
    
    if len(na_df) > 0:
        print("Following columns have NA values:\n")
        
        for k, v in na_df.items():
            print(k, v)
    else:
        print("No NA values!")

def merge_dataset_and_labels(df: pd.DataFrame, labels: pd.DataFrame) -> pd.DataFrame:
    df["week"] = df["video_name"].map(get_week_number)
    df = df[df["week"] == 6]
    df["trial_number"] = df["video_name"].map(get_trial_number)
    df["set"] = df["video_name"].map(get_video_set)
    df = df.merge(labels, how="inner", on=["set", "trial_number"])
    df.drop(columns=["video_name", "week_x", "week_y", "trial_number", "set", "VelocityInner_Total"], inplace=True)
    
    return df

def save_new_dataset(dataset_path: str, result_path: str, experiment="EPM"):
    labels = pd.read_excel(LABELS_PATH, engine="openpyxl")
    labels = labels[labels["test"] == experiment]
    labels = preprocess_labels(labels)
    
    df = pd.read_csv(dataset_path)
    df = merge_dataset_and_labels(df, labels)
    df.to_csv(result_path)

def plot_labels_distribution(df: pd.DataFrame, experiment_name: str):
    targets = ["High_Anxiety", "Middle_Anxiety", "Low_Anxiety"]
    counts = df[targets].sum().values
    
    plt.figure(figsize=(8, 8))
    plt.bar(targets, counts, color=["red", "yellow", "green"])
    plt.title(f"Labels distribution for the {experiment_name}", fontsize=16)
    plt.ylabel("Frequency", fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(axis="y")
    plt.show()

In [14]:
def read_labels(labels_path):
    labels = pd.read_excel(labels_path, engine="openpyxl")
    labels = labels[labels["test"] == EXPERIMENT]
    labels = preprocess_labels(labels)
    return labels

def get_test_labels(labels, train_labels):
    test_labels = pd.merge(train_labels, labels, on=["set", "week", "trial_number"], how="right", indicator=True).loc[lambda x: x["_merge"] == "right_only"]
    test_labels.drop(columns=["High_Anxiety_x", "Middle_Anxiety_x", "Low_Anxiety_x", "_merge"], inplace=True)
    test_labels.rename(columns={
        "High_Anxiety_y": "High_Anxiety", 
        "Middle_Anxiety_y": "Middle_Anxiety", 
        "Low_Anxiety_y": "Low_Anxiety"
    }, inplace=True)
    return test_labels

def read_dataset(dataset_path, labels):
    df = pd.read_csv(dataset_path)
    df = merge_dataset_and_labels(df, labels)
    return df

def get_labels_dataset(df):
    y = df[LABELS]
    y = [2 - np.argmax(row) for row in y.values]

    df.drop(columns=LABELS, inplace=True)
    df.head()
    return y
    
def get_dataset():
    all_labels = read_labels(LABELS_PATH)
    X = read_dataset(DATASET_PATH, all_labels)
    y = get_labels_dataset(X)
    
    return X, y

In [15]:
X, y = get_dataset()

In [16]:
X.describe()

,DistanceTraveled_Total,TimeCenter_Total,TimeOuter_Total,DistanceInner_Total,DistanceOuter_Total,Velocity_Total,VelocityOuter_Total,FrequencyToEnterZones_Total,Immobility_Total,LatencyToFirstEnter_Total,...,running_3000,running_4500,running_6000,running_7500,running_9000,running_10500,running_12000,running_13500,running_15000,RunTurn
count,105.000000,105.000000,105.00000,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,...,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000,105.000000
mean,45736.167114,139.587429,463.16419,16848.101709,28888.065404,75.878829,60.845414,121.838095,318.131048,32.277714,...,0.771429,0.771429,0.714286,0.771429,0.733333,0.695238,0.695238,0.714286,0.752381,7.428571
std,16521.352643,112.586379,112.60154,12451.923033,15176.587458,27.409147,23.317450,84.163431,82.164018,67.365244,...,0.421927,0.421927,0.453921,0.421927,0.444338,0.462514,0.462514,0.453921,0.433699,3.210132
min,12933.285517,17.040000,134.36000,3130.479585,4517.798282,21.456775,20.828784,19.000000,182.760000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,32797.253004,42.880000,403.40000,6492.261584,17321.924341,54.408183,42.161542,57.000000,260.320000,0.920000,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000
50%,49324.144229,115.680000,487.08000,10951.048751,25279.810091,81.814199,60.944544,80.000000,300.640000,7.440000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000
75%,56997.007573,199.360000,559.96000,27409.366853,40315.346772,94.591423,76.193884,181.000000,374.760000,19.560000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000
max,80094.955171,468.400000,585.84000,42712.096457,72019.900358,132.853893,125.697955,322.000000,514.120000,407.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000


In [17]:
X

,DistanceTraveled_Total,TimeCenter_Total,TimeOuter_Total,DistanceInner_Total,DistanceOuter_Total,Velocity_Total,VelocityOuter_Total,FrequencyToEnterZones_Total,Immobility_Total,LatencyToFirstEnter_Total,...,running_3000,running_4500,running_6000,running_7500,running_9000,running_10500,running_12000,running_13500,running_15000,RunTurn
0,45720.013036,24.36,578.48,4343.077734,41376.935302,75.841041,71.526994,40,347.20,50.28,...,1,1,1,1,1,1,0,1,1,9
1,45720.013036,24.36,578.48,4343.077734,41376.935302,75.841041,71.526994,40,347.20,50.28,...,1,1,1,1,1,1,0,1,1,9
2,12933.285517,468.40,134.36,8415.487235,4517.798282,21.456775,33.624578,46,514.12,76.64,...,0,0,0,0,0,0,0,0,1,1
3,12933.285517,468.40,134.36,8415.487235,4517.798282,21.456775,33.624578,46,514.12,76.64,...,0,0,0,0,0,0,0,0,1,1
4,32797.253004,251.84,350.96,19093.908772,13703.344232,54.408183,39.045316,139,375.80,1.32,...,1,0,1,1,1,0,0,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,39241.449319,21.28,581.60,3130.479585,36110.969733,65.089984,62.089013,36,351.92,97.40,...,1,1,1,1,1,1,1,0,1,8
101,33600.116459,342.24,260.56,18810.168759,14789.947700,55.740074,56.762157,95,396.56,15.12,...,1,1,0,0,0,0,0,0,0,3
102,33600.116459,342.24,260.56,18810.168759,14789.947700,55.740074,56.762157,95,396.56,15.12,...,1,1,0,0,0,0,0,0,0,3
103,56970.621924,99.96,502.80,26327.705198,30642.916726,94.516262,60.944544,196,282.28,0.00,...,1,1,1,1,1,1,1,1,1,10


In [18]:
check_na(X)

Following columns have NA values:

VelocityInner_1500 14
VelocityInner_3000 16
VelocityInner_4500 10
VelocityInner_6000 8
VelocityInner_7500 16
VelocityInner_9000 14
VelocityInner_10500 6
VelocityInner_12000 16
VelocityInner_13500 10
VelocityInner_15000 12
VelocityOuter_1500 2
VelocityOuter_4500 2
VelocityOuter_6000 2
VelocityOuter_7500 2
VelocityOuter_9000 2
VelocityOuter_10500 2
VelocityOuter_12000 2
VelocityOuter_13500 2


In [19]:
# Temporarily remove NA columns

X.dropna(axis=1, inplace=True)

check_na(X)

No NA values!


In [20]:
X

,DistanceTraveled_Total,TimeCenter_Total,TimeOuter_Total,DistanceInner_Total,DistanceOuter_Total,Velocity_Total,VelocityOuter_Total,FrequencyToEnterZones_Total,Immobility_Total,LatencyToFirstEnter_Total,...,running_3000,running_4500,running_6000,running_7500,running_9000,running_10500,running_12000,running_13500,running_15000,RunTurn
0,45720.013036,24.36,578.48,4343.077734,41376.935302,75.841041,71.526994,40,347.20,50.28,...,1,1,1,1,1,1,0,1,1,9
1,45720.013036,24.36,578.48,4343.077734,41376.935302,75.841041,71.526994,40,347.20,50.28,...,1,1,1,1,1,1,0,1,1,9
2,12933.285517,468.40,134.36,8415.487235,4517.798282,21.456775,33.624578,46,514.12,76.64,...,0,0,0,0,0,0,0,0,1,1
3,12933.285517,468.40,134.36,8415.487235,4517.798282,21.456775,33.624578,46,514.12,76.64,...,0,0,0,0,0,0,0,0,1,1
4,32797.253004,251.84,350.96,19093.908772,13703.344232,54.408183,39.045316,139,375.80,1.32,...,1,0,1,1,1,0,0,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,39241.449319,21.28,581.60,3130.479585,36110.969733,65.089984,62.089013,36,351.92,97.40,...,1,1,1,1,1,1,1,0,1,8
101,33600.116459,342.24,260.56,18810.168759,14789.947700,55.740074,56.762157,95,396.56,15.12,...,1,1,0,0,0,0,0,0,0,3
102,33600.116459,342.24,260.56,18810.168759,14789.947700,55.740074,56.762157,95,396.56,15.12,...,1,1,0,0,0,0,0,0,0,3
103,56970.621924,99.96,502.80,26327.705198,30642.916726,94.516262,60.944544,196,282.28,0.00,...,1,1,1,1,1,1,1,1,1,10


## Temporary experiments

In [28]:
main_columns = [
    "DistanceTraveled_Total",
    "TimeCenter_Total",
    "TimeOuter_Total",
    "DistanceInner_Total",
    "DistanceOuter_Total",
    "Velocity_Total",
    "VelocityOuter_Total",
    "FrequencyToEnterZones_Total",
    "Immobility_Total",
    "LatencyToFirstEnter_Total",
    "TurnRights_Total",
    "TurnLeft_Total",
    "RunTurn"
]

X = X[main_columns]
X

,DistanceTraveled_Total,TimeCenter_Total,TimeOuter_Total,DistanceInner_Total,DistanceOuter_Total,Velocity_Total,VelocityOuter_Total,FrequencyToEnterZones_Total,Immobility_Total,LatencyToFirstEnter_Total,TurnRights_Total,TurnLeft_Total,RunTurn
0,45720.013036,24.36,578.48,4343.077734,41376.935302,75.841041,71.526994,40,347.20,50.28,3716,3817,9
1,45720.013036,24.36,578.48,4343.077734,41376.935302,75.841041,71.526994,40,347.20,50.28,3716,3817,9
2,12933.285517,468.40,134.36,8415.487235,4517.798282,21.456775,33.624578,46,514.12,76.64,3800,3732,1
3,12933.285517,468.40,134.36,8415.487235,4517.798282,21.456775,33.624578,46,514.12,76.64,3800,3732,1
4,32797.253004,251.84,350.96,19093.908772,13703.344232,54.408183,39.045316,139,375.80,1.32,3800,3733,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,39241.449319,21.28,581.60,3130.479585,36110.969733,65.089984,62.089013,36,351.92,97.40,3885,3649,8
101,33600.116459,342.24,260.56,18810.168759,14789.947700,55.740074,56.762157,95,396.56,15.12,3847,3686,3
102,33600.116459,342.24,260.56,18810.168759,14789.947700,55.740074,56.762157,95,396.56,15.12,3847,3686,3
103,56970.621924,99.96,502.80,26327.705198,30642.916726,94.516262,60.944544,196,282.28,0.00,3815,3717,10


In [234]:
import pickle

# model = RandomForestClassifier(min_samples_leaf=7, n_estimators=13, random_state=0, n_jobs=-1)

# vals = cross_val_score(model, X, y, scoring="accuracy", n_jobs=-1, cv=5)
# np.mean(vals), vals

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, random_state=0, test_size=0.2)
model = pickle.load(open("OF_classification_model.sav", "rb"))
# model.fit(X_train, y_train)
list(model.predict(X_valid)), y_valid

([1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 0, 1, 2, 0, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 2])

## Independent Multiclass Classification

In [31]:
def calculate_metrics(model, X, y, metrics):
    skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=0)
    
    scorings = {}
    
    for scoring in metrics:
        if scoring == "neg_root_mean_squared_error":
            sign = -1
        else:
            sign = 1
            
        skf_scores = cross_val_score(model, X, y, cv=skf, scoring=scoring)

        scorings[scoring] = np.round(sign * np.mean(skf_scores), 3)
    
    return scorings

In [32]:
def data_preprocessing(X, y, preprocessing, n_components):
    datasets = {}
    temp_X = X.copy()
    
    if len(preprocessing) == 0:
        print("Error! No preprocessing techniques were provided. Available techniques are: selectkbset, pca, umap, total.")
    
    for technique in preprocessing:
        if technique == "selectkbest":
            selector = SelectKBest(chi2, k=n_components)
            temp_X   = selector.fit_transform(temp_X.replace(-1, 999), y)
            
        scaler = StandardScaler()
        temp_X = scaler.fit_transform(temp_X)
        
        if technique == "pca":
            pca    = PCA(n_components=n_components, random_state=0)
            temp_X = pca.fit_transform(temp_X)
        elif technique == "umap":
            UMAP   = umap.UMAP(n_components=n_components, random_state=0, n_jobs=-1)
            temp_X = UMAP.fit_transform(temp_X)
        elif technique != "selectkbest":
            continue
        
        datasets[technique] = temp_X
        
    return datasets

In [33]:
def test_model(model, X: pd.DataFrame, y, n_components=10, preprocessing=["selectkbest", "pca", "umap"], 
               metrics=["accuracy", "neg_root_mean_squared_error"]) -> pd.DataFrame:
    results = {}
    all_scores = {}
    
    results = pd.DataFrame(columns=[
        "n_components", 
        "dataset_type", 
        "stratified-kfold acc",
        "stratified-kfold rmse"
    ])
    
    for n in range(1, n_components+1):
        datasets = data_preprocessing(X, y, preprocessing, n)
        
        for name, dataset in datasets.items():
            scores = calculate_metrics(model, dataset, y, metrics)
            row_values = [n, name]
            
            for metric, score in scores.items():
                row_values.append(score)
            
            results = results.append({k: v for k, v in zip(results, row_values)}, ignore_index=True)
                
        print(f"Number of components finished: {n} out of {n_components}", end='\r')
                    
    return results

In [34]:
class OrdinalClassifier:
    def __init__(self, **kwargs):
        self.clf = LogisticRegression(**kwargs)
        self.clfs = {}
        
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                binary_y = (y > self.unique_class[i]).astype(np.int8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
                
    def predict_proba(self, X):
        clfs_predict = {k: self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        
        for i, y in enumerate(self.unique_class):
            if i == 0:
                predicted.append(1 - clfs_predict[y][:, 1])
            elif y in clfs_predict:
                predicted.append(clfs_predict[y-1][:, 1] - clfs_predict[y][:, 1])
            else:
                predicted.append(clfs_predict[y-1][:, 1])
                
        return np.vstack(predicted).T
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def score(self, X, y):
        return accuracy_score(y, self.predict(X))
    
    def get_params(self, deep=True):
        out = self.clf.get_params(deep=deep)
        return out

In [35]:
# best: 0.476, 71.8%, 2, selectkbest or pca (default)
model = OrdinalClassifier(n_jobs=-1, random_state=0)

results = test_model(model, X, y)
results.sort_values(["stratified-kfold rmse", "stratified-kfold acc", "n_components"], ascending=True)

,n_components,dataset_type,stratified-kfold acc,stratified-kfold rmse
3,2,selectkbest,0.718,0.476
4,2,pca,0.718,0.476
6,3,selectkbest,0.718,0.476
7,3,pca,0.718,0.476
9,4,selectkbest,0.694,0.495
10,4,pca,0.694,0.495
21,8,selectkbest,0.697,0.499
22,8,pca,0.697,0.499
24,9,selectkbest,0.697,0.499
25,9,pca,0.697,0.499


In [20]:
# best: 0.418, 74.1%, 4, selectkbest or pca (default)
model = LogisticRegression(n_jobs=-1, random_state=0)

results = test_model(model, X, y)
results.sort_values(["stratified-kfold rmse", "stratified-kfold acc", "n_components"], ascending=True)

,n_components,dataset_type,stratified-kfold acc,stratified-kfold rmse
8,3,umap,0.600,0.650
12,5,selectkbest,0.571,0.651
13,5,pca,0.571,0.651
9,4,selectkbest,0.562,0.652
10,4,pca,0.562,0.652
15,6,selectkbest,0.552,0.664
16,6,pca,0.552,0.664
18,7,selectkbest,0.533,0.679
19,7,pca,0.533,0.679
2,1,umap,0.533,0.683


In [16]:
# best: 0.593, 62.2%, 2, selectkbest or pca (default)
model = RidgeClassifier(random_state=0)

results = test_model(model, X, y)
results.sort_values(["stratified-kfold rmse", "stratified-kfold acc", "n_components"], ascending=True)

,n_components,dataset_type,stratified-kfold acc,stratified-kfold rmse
0,1,selectkbest,0.543,0.675
1,1,pca,0.543,0.675
2,1,umap,0.533,0.683
9,4,selectkbest,0.524,0.687
10,4,pca,0.524,0.687
12,5,selectkbest,0.524,0.687
13,5,pca,0.524,0.687
15,6,selectkbest,0.514,0.694
16,6,pca,0.514,0.694
5,2,umap,0.514,0.696


In [17]:
# best: 0.394, 76.5%, 2, umap (kernel="poly", degree=4)
model = SVC(kernel="poly", degree=7, random_state=0)

results = test_model(model, X, y)
results.sort_values(["stratified-kfold rmse", "stratified-kfold acc", "n_components"], ascending=True)

,n_components,dataset_type,stratified-kfold acc,stratified-kfold rmse
5,2,umap,0.600,0.652
3,2,selectkbest,0.571,0.654
4,2,pca,0.571,0.654
0,1,selectkbest,0.543,0.676
1,1,pca,0.543,0.676
2,1,umap,0.533,0.683
8,3,umap,0.524,0.687
6,3,selectkbest,0.552,0.688
7,3,pca,0.552,0.688
24,9,selectkbest,0.524,0.689


In [18]:
# best: 0.399, 77.2%, 3, pca (default)
model = DecisionTreeClassifier(random_state=0)

results = test_model(model, X, y)
results.sort_values(["stratified-kfold rmse", "stratified-kfold acc", "n_components"], ascending=True)

,n_components,dataset_type,stratified-kfold acc,stratified-kfold rmse
20,7,umap,0.476,0.760
5,2,umap,0.467,0.783
14,5,umap,0.514,0.783
26,9,umap,0.457,0.785
2,1,umap,0.438,0.799
23,8,umap,0.429,0.816
29,10,umap,0.419,0.828
11,4,umap,0.333,0.849
0,1,selectkbest,0.390,0.880
1,1,pca,0.390,0.880


In [19]:
# best: 0.382, 79.3%, 9, selectkbest or pca (n_neighbors=1)
model = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)

results = test_model(model, X, y)
results.sort_values(["stratified-kfold rmse", "stratified-kfold acc", "n_components"], ascending=True)

,n_components,dataset_type,stratified-kfold acc,stratified-kfold rmse
2,1,umap,0.438,0.799
5,2,umap,0.448,0.811
8,3,umap,0.486,0.826
23,8,umap,0.400,0.834
11,4,umap,0.371,0.837
29,10,umap,0.410,0.854
17,6,umap,0.438,0.864
20,7,umap,0.381,0.868
26,9,umap,0.400,0.871
6,3,selectkbest,0.400,0.874


In [58]:
# best: 0.401, 76.9%, 6, pca (default)
model = RandomForestClassifier(n_jobs=-1, random_state=0)

results = test_model(model, X, y)
results.sort_values(["stratified-kfold rmse", "stratified-kfold acc", "n_components"], ascending=True)

,n_components,dataset_type,stratified-kfold acc,stratified-kfold rmse
16,6,pca,0.769,0.401
22,8,pca,0.724,0.442
19,7,pca,0.745,0.459
7,3,pca,0.680,0.472
11,4,umap,0.677,0.476
6,3,selectkbest,0.718,0.479
15,6,selectkbest,0.721,0.483
29,10,umap,0.721,0.483
8,3,umap,0.697,0.499
18,7,selectkbest,0.701,0.500
